In [ ]:
import os
from experiment_setup import setups

In [ ]:
# download the dataset, the links can be fund in the README
dataset_path = "./datasets/coveo_ecommerce"
model_path = "./trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [ ]:
params = setups["coveo"]["params_xe"]

In [ ]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [ ]:
def create_gru4rec_tensorflow_script(model_name, train_path, test_path, model_path, loss, final_act, layers, batch_size, dropout_p_hidden, learning_rate, hidden_act, n_epochs, m, decay, initial_accumulator):
        """Creates the train and test script.
        """
        s_train_full = f"CUDA_VISIBLE_DEVICES=0 python ./GRU4Rec_TensorFlow/gru4rec_BP/main.py --train_path {train_path} --test_path {test_path} --checkpoint_dir {model_path}/{model_name} --layer {1} --size {layers} --epoch {n_epochs} --lr {learning_rate} --hidden_act {hidden_act} --final_act {final_act} --loss {loss} --dropout {dropout_p_hidden} --batch_size {batch_size} --initial_accumulator_value {initial_accumulator} --decay {decay}"
        s_test_full = s_train_full + f" --train 0 --test {n_epochs-1} --m {m}"
        return s_train_full, s_test_full

In [ ]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [ ]:
train_script_oob, test_script_oob = create_gru4rec_tensorflow_script(model_name="gru4rec_tensorflow_oob", train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=50, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=0.96, initial_accumulator=1e-1)

### Train the out-of-the-box model

In [ ]:
os.system(train_script_oob)

### Test the out-of-the-box model

In [ ]:
os.system(test_script_oob)

## Train & test the minor fix model


In [ ]:
train_script_minor, test_script_minor = create_gru4rec_tensorflow_script(model_name='gru4rec_tensorflow_minorfix', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, hidden_act=hidden_act, n_epochs=n_epochs, m=m, decay=1.0, initial_accumulator=1e-12)

### Train the minor fix model

In [ ]:
os.system(train_script_minor)

### Test the minor fix model

In [ ]:
os.system(test_script_minor)